In [1]:
!pip install rouge
import torch
import torch.nn as nn
import numpy as np
import rouge
from transformers import AutoConfig, AutoTokenizer, MBartForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
#from utils.preprocess import *
from rouge import Rouge     
import wandb
import os

2024-04-21 15:55:58.473719: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 15:55:58.473823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 15:55:58.593067: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tokenizer = AutoTokenizer.from_pretrained('vinai/bartpho-word-base')
model = MBartForConditionalGeneration.from_pretrained('vinai/bartpho-word-base')
#tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/600M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
def compute_rouge1_fmeasure_score_vlsp(preds, target):
    rouge = Rouge()
    scores = rouge.get_scores(preds, target, avg=False)
    rouge1_fmeasure= scores[0]['rouge-1']["f"]
    rouge2_fmeasure= scores[0]['rouge-2']["f"]
    rougeL_fmeasure= scores[0]['rouge-l']["f"]
    return {
        'rouge_1': rouge1_fmeasure,
        'rouge_2': rouge2_fmeasure,
        'rouge_L': rougeL_fmeasure,
    }
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = compute_rouge1_fmeasure_score_vlsp(preds=decoded_preds, target=decoded_labels)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result['gen_len'] = np.mean(prediction_lens)

    return {
        'rouge-1': result['rouge_1'],
        'rouge_2': result['rouge_2'],
        'rouge_L': result['rouge_L'],
        'gen_len': result['gen_len'],
    }

In [4]:
def preprocess_function(sample):
    # Tạo input cho mô hình từ câu hỏi và nội dung
    inputs = sample["question"] + " " + sample["context"]
    # Tạo nhãn cho mô hình từ câu trả lời
    #labels = sample["answer"]


    # tokenize inputs
    #model_inputs = tokenizer(inputs, max_length=1024, padding='max_length', truncation=True)
    model_inputs = tokenizer(
        sample["question"],
        sample["context"],
        max_length=1024,
        truncation= "only_second",
        stride=50,
        padding = True
        #return_overflowing_tokens=True,
        #return_offsets_mapping=True,
        )
    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["answer"], max_length=1024, padding='max_length', truncation=True)

    model_inputs["labels"] = labels["input_ids"]
 
    return {
        'input_ids': model_inputs['input_ids'],
        'attention_mask': model_inputs['attention_mask'],
        'labels': model_inputs['labels']
    }

In [5]:
from datasets import Dataset
import json
def merge_json_files(file1_path, file2_path):
    # Đọc dữ liệu từ file 1
    with open(file1_path, 'r', encoding='utf-8') as file1:
        data1 = json.load(file1)
    
    # Đọc dữ liệu từ file 2
    with open(file2_path, 'r', encoding='utf-8') as file2:
        data2 = json.load(file2)
    
    merged_data = []
    x = len(data2)//len(data1)
    # Nối dữ liệu từ file 1 và file 2, chỉ giữ lại các trường "question", "context", và "answer"
    i, j = 0, 0
    while i < len(data1) and j < len(data2):
        for _ in range(x):
            if j < len(data2):
                merged_data.append(data2[j])
                j += 1
        merged_data.append(data1[i])
        i += 1

    # Nếu còn dữ liệu ở file 2, thêm vào cuối
    while j < len(data2):
        merged_data.append(data2[j])
        j += 1

    return merged_data

In [6]:
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [7]:
# Đọc dữ liệu từ file JSON
train_data = load_json_file('/kaggle/input/law-qa/Law-QA-train.json')
test_data = load_json_file('/kaggle/input/law-qa/Law-QA-test.json')

columns = ["question", "context", "answer"]

# Tạo danh sách dữ liệu của các cột
list_data_train = {column: [item[column] for item in train_data] for column in columns}
list_data_test = {column: [item[column] for item in test_data] for column in columns}
# Tạo đối tượng Dataset từ danh sách dữ liệu
train = Dataset.from_dict(list_data_train)
test = Dataset.from_dict(list_data_test)

In [8]:
def train_test_splits():
    train_data = (train.map(preprocess_function, remove_columns=['question', 'context', 'answer']))
    test_data = (test.map(preprocess_function, remove_columns=['question', 'context', 'answer']))
    return train_data, test_data

print('------------------------------------------------Loading Data----------------------------------------------------')
train_data, test_data = train_test_splits()
print('----------------------------------------------------Done!-------------------------------------------------------')

------------------------------------------------Loading Data----------------------------------------------------


  0%|          | 0/168000 [00:00<?, ?ex/s]

  0%|          | 0/2898 [00:00<?, ?ex/s]

----------------------------------------------------Done!-------------------------------------------------------


In [9]:
from torch.utils.data import Subset
subset_size = 144000
suset_size = 200
subset_train_data = Subset(train_data, range(len(train_data) - subset_size, len(train_data)))
subset_valid_data = Subset(test_data, range(suset_size))

In [10]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    pad_to_multiple_of=8,
    #padding = True
)

training_args =Seq2SeqTrainingArguments(
    output_dir="./Checkpoint Bartpho",
    gradient_checkpointing=True,
    do_train=True,
    remove_unused_columns=False,
    warmup_ratio=0.05,
    weight_decay=0.01,
    group_by_length=True,
    fp16=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    predict_with_generate=True,
    learning_rate=2e-5,
    num_train_epochs=1,
    logging_strategy="steps",
    logging_steps=3000,
    save_steps=3000,
    eval_steps=3000,
    evaluation_strategy="steps",
    save_total_limit=5,
)



trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    eval_dataset=test_data,
    train_dataset=subset_train_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# tokenizer.save_pretrained('./Checkpoint Bartpho/Final/Tokenizer-Pipeline')
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge-1,Rouge 2,Rouge L,Gen Len
3000,0.370700,0.116160,0.333333,0.121951,0.166667,19.873706
6000,0.120500,0.100153,0.333333,0.121951,0.166667,19.891994
9000,0.108900,0.093353,0.333333,0.121951,0.166667,19.885438


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer

TrainOutput(global_step=9000, training_loss=0.2000576477050781, metrics={'train_runtime': 38259.4018, 'train_samples_per_second': 3.764, 'train_steps_per_second': 0.235, 'total_flos': 6.932390312293171e+16, 'train_loss': 0.2000576477050781, 'epoch': 1.0})

In [11]:
#trainer.evaluate()